In [116]:
import re
from datetime import datetime

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)
from pandas.io.json import json_normalize
import json
import ast

%load_ext autotime

import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


time: 411 ms


In [117]:
commits = pd.read_csv('data/commits.csv', parse_dates=True)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning:

Columns (22,23) have mixed types. Specify dtype option on import or set low_memory=False.



time: 16.7 s


In [118]:
commits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26042 entries, 0 to 26041
Data columns (total 62 columns):
Unnamed: 0                       26042 non-null int64
sha                              26042 non-null object
node_id                          26042 non-null object
url                              26042 non-null object
html_url                         26042 non-null object
comments_url                     26042 non-null object
parents                          26042 non-null object
repo_name                        26042 non-null object
owner                            26042 non-null object
commit.author.name               26042 non-null object
commit.author.email              26041 non-null object
commit.author.date               26042 non-null object
commit.committer.name            26042 non-null object
commit.committer.email           26042 non-null object
commit.committer.date            26042 non-null object
commit.message                   26042 non-null object
commit.tree.

## Data Preprocessing

In [119]:
commits['date'] =  pd.to_datetime(commits['commit.committer.date'])

time: 217 ms


In [120]:
commits['date'] =  pd.to_datetime(commits['date'], utc=True)

time: 35 ms


In [121]:
commits['commit_date'] = commits['date'].dt.date

time: 50 ms


In [122]:
commits['commit_week'] = commits['date'].dt.week

time: 49 ms


In [123]:
commits['commit_hour'] = commits['date'].dt.hour

time: 7 ms


In [124]:
commits['commit_month'] = commits['date'].dt.month

time: 10 ms


In [125]:
commits['commit_year'] = commits['date'].dt.year

time: 9 ms


In [126]:
# drop unnecessary columns
commits = commits[['sha', 'author.login', 'commit_date', 'commit_hour', 'commit_month', 'commit_year']]

time: 392 ms


In [127]:
commits.head()

,sha,author.login,commit_date,commit_hour,commit_month,commit_year
0,ba3f6330dd2b6054988f1f6f0ffe014fc4969088,cloud-fan,2019-12-24,6,12,2019
1,ab0dd41ff2c05134266bd14d96676c96aa2764cd,MaxGekk,2019-12-24,3,12,2019
2,2164243526b5a2ebd1a77fe2aa43b7329b6a229c,HeartSaVioR,2019-12-24,2,12,2019
3,7bff2db9ed803e05a43c2d875c1dea819d81248a,HeartSaVioR,2019-12-23,22,12,2019
4,c6ab7165dd11a0a7b8aea4c805409088e9a41a74,mobai-zhanjf,2019-12-23,16,12,2019


time: 130 ms


## Data Analysis

In [128]:
commits['author.login'].unique().size

1456

time: 14 ms


In [129]:
commits_by_hour = commits.groupby('commit_hour')[['sha']].count()
commits_by_hour = commits_by_hour.rename(columns = {'sha': 'commit_count'})

time: 111 ms


In [130]:
fig = go.Figure([go.Bar(
    x=commits_by_hour.index, 
    y=commits_by_hour.commit_count, 
    text=commits_by_hour.commit_count, 
    textposition='auto')])
fig.update_layout(
    title = 'Commits by Hour', 
    xaxis_title = 'Hour', 
    yaxis_title = 'Commits Count', 
    xaxis_tickmode = 'linear')
fig.show()

time: 2.27 s


In [131]:
commits_by_day = commits.groupby('commit_date')[['sha']].count()
commits_by_day = commits_by_day.rename(columns = {'sha': 'commit_count'})

time: 38 ms


In [132]:
fig = go.Figure([go.Scatter(
    x=commits_by_day.index, 
    y=commits_by_day.commit_count, 
    text=commits_by_day.commit_count, 
    fill='tozeroy')])
fig.update_layout(
    title = 'Commits by Date', 
    xaxis_title = 'Date', 
    yaxis_title = 'Commits Count')
fig.show()

time: 11 s


In [189]:
yearly_commits = commits.groupby('commit_year')[['sha']].count()
yearly_commits = yearly_commits.rename(columns = {'sha': 'commits_count'})
yearly_commits['daily_avg'] = yearly_commits['commits_count']/365

time: 800 ms


In [195]:
fig = go.Figure([go.Scatter(
    x=yearly_commits.index, 
    y=yearly_commits.daily_avg, 
    text=yearly_commits.daily_avg
)])
fig.update_layout(
    title = 'Average Daily Commits by Year', 
    xaxis_title = 'Year', 
    yaxis_title = 'Average Number of Commits',
    xaxis_tickmode = 'linear')
fig.show()

time: 378 ms


In [96]:
commits_by_author = commits.groupby('author.login')[['sha']].count()
commits_by_author = commits_by_author.rename(columns = {'sha': 'commit_count'})
commits_by_author = commits_by_author.sort_values(by='commit_count', ascending=False)
top_authors = commits_by_author.head(30)

time: 22.5 ms


In [97]:
fig = go.Figure([go.Bar(
    x=top_authors.index, 
    y=top_authors.commit_count)])
fig.update_layout(
    title = 'Top Committers', 
    xaxis_title = 'Author', 
    yaxis_title = 'Commits Count', 
    xaxis_tickmode = 'linear',
    xaxis_tickangle=-40)
fig.show()

time: 315 ms


## Open Pull Requests

In [98]:
pulls = pd.read_csv('data/pulls.csv', parse_dates=True)

time: 258 ms


In [99]:
pulls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521 entries, 0 to 520
Columns: 289 entries, Unnamed: 0 to head.repo
dtypes: bool(14), float64(24), int64(16), object(235)
memory usage: 1.1+ MB
time: 82.5 ms


In [100]:
pulls['date'] = pd.to_datetime(pulls['created_at'])
pulls['date'] = pd.to_datetime(pulls['date'], utc=True)
pulls['pull_date'] = pulls['date'].dt.date

time: 25 ms


In [101]:
pulls_by_date = pulls.groupby('pull_date')[['id']].count()
pulls_by_date = pulls_by_date.rename(columns = {'id': 'commit_count'})

time: 25 ms


In [104]:
fig = go.Figure([go.Scatter(
    x=pulls_by_date.index, 
    y=pulls_by_date.commit_count, 
    text=pulls_by_date.commit_count)])
fig.update_layout(
    title = 'Open Pull Requests by Date', 
    xaxis_title = 'Date', 
    yaxis_title = 'Pulls Count')
fig.show()

time: 325 ms


### Pull Request Labels

**NOTES**:  
- ast.literal_eval converts string to list.
- The following two lines convert a Pandas column that contains list of dictionaries into a Pandas Dataframe where each dictionary corresponds to a dataframe row.

In [105]:
labels_list = [ast.literal_eval(i) for i in pulls['labels'].tolist() if i != '[]']

time: 82.5 ms


In [106]:
labels = [j for i in labels_list for j in i]

time: 2.5 ms


In [108]:
labels_df = pd.DataFrame(labels)

time: 10 ms


In [114]:
labels_by_name = labels_df.groupby('name')[['id']].count()
labels_by_name = labels_by_name.rename(columns = {'id': 'label_count'})
labels_by_name = labels_by_name.sort_values(by=['label_count'], ascending=False)

time: 7.5 ms


In [115]:
fig = go.Figure([go.Bar(
    x=labels_by_name.index, 
    y=labels_by_name.label_count)])
fig.update_layout(
    title = 'Pull Requests by Label', 
    xaxis_title = 'Labels', 
    yaxis_title = 'PR Count', 
    xaxis_tickmode = 'linear',
    xaxis_tickangle=-40)
fig.show()

time: 248 ms
